This notebook explains how to make predictions for Google Street View images using Yolo v3 and ImageAI library with the help of this [tutorial](https://github.com/OlafenwaMoses/ImageAI/tree/master/imageai/Detection).


In [ ]:
!pip install opencv-python==4.1.2.30 imageai
!pip install requests
from google.colab.patches import cv2_imshow
import os
import cv2 as cv
from imageai.Detection import ObjectDetection
from collections import Counter
import pandas as pd
import numpy as np
import requests as req
import os as os

     |████████████████████████████████| 163kB 18.2MB/s 
     |████████████████████████████████| 2.1MB 35.7MB/s 
     |████████████████████████████████| 11.6MB 241kB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=bfd0ac4844597cdd78f235432acbd86f52780d24e51cb2c09dc6a279636824ca
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-resnet
ERROR: bokeh 2.3.1 has requirement pillow>=7.1.0, but you'll have pillow 7.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: imageai 2.1.6 has requirement numpy==1.19.3, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:


In [ ]:
# At our work images were uploaded into Google Drive so we mouth here the drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Some utility functions.

def showImage(img):
    window_name = 'image'
    cv2_imshow(img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
#TO DO: This part will be filled with your input path for images.
input_path = ''

In [ ]:
# Here we will list links to images we are going to detect objects.
test_images = os.listdir(input_path)

In [ ]:
# Getting our YOLOv3 model.
modelYOLOv3 = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5'

if not os.path.exists('yolo.h5'):
    r = req.get(modelYOLOv3, timeout=0.5)
    with open('yolo.h5', 'wb') as outfile:
        outfile.write(r.content)

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('yolo.h5')
detector.loadModel()

In [ ]:
# Predicting for images. This may take sometime depending on number of images.
results = []
bounded_images = []
for i in test_images:
  detectedImage, detections = detector.detectObjectsFromImage(output_type="array", input_image=input_path + i, minimum_percentage_probability=30)
  convertedImage = cv.cvtColor(detectedImage, cv.COLOR_RGB2BGR)
  results.append(detections)
  bounded_images.append(convertedImage)


In [ ]:
# Counting images in each item and writing into dataframe.
all_items = []
final_counts=[]
new_item = {}
for i in range(len(results)):
  result_dict = results[i]
  for z in range(len(result_dict)):
    image_objects= []
    new_item[z] = result_dict[z]['name']
    image_objects.append(new_item)
    number_of_items = Counter(image_objects[0].values())
    image_dict  = {'image_name': test_images[i], 'no_of_items':number_of_items}
    final_counts.append(image_dict)

In [ ]:
# Finalizing into dataframe.
final_df=pd.DataFrame(columns=['index','image_name','no_of_items'])
for i in range(len(final_counts)):
  df =   pd.DataFrame(final_counts[i]).reset_index()
  final_df =pd.concat([final_df,df],axis=0)


In [ ]:
# Showing example predictions. You can change the images you would like to show by changing the number below.
showImage(bounded_images[1])
